# Explicação rápida dos dados (resumo):

- Registros: clientes (customer_id) com variáveis demográficas, comportamentais, transacionais, datas, texto e target churn.
- Numéricas: age, income (forte skew + outliers), tenure_months, num_transactions, avg_transaction. 
    - skew se refere a desequilibrios ou distorções na distribuição.
    - outliers são valores extremos que podem distorcer análises estatísticas. Em termos gerais, são dados que fogem do padrão esperado.
- Categóricas: country, product, device (BR tem peso maior).
- Datas: signup_date, last_login — permitem criar recência, idade da conta, frequência.
- Texto: review_text (comentários curtos, útil para TF-IDF/embeddings). -> análise de sentimentos, tópicos.
- Target: churn (binário, desequilibrado — ~10% base). -> prever abandono.
- Problemas artificiais incluídos: missing (income, last_login, review_text), outliers extremos em income/avg_transaction, duplicatas, distribuição enviesada.
    - Esses problemas simulam desafios reais em dados de clientes.

# Tarefa proposta (prática de feature engineering + baseline):

### Objetivo: montar um pipeline reprodutível que faça engenharia de features, treine um modelo simples e reporte métricas.

> Passos mínimos (sugestão):

1. Exploração
--------------------------------------------------------------------------------------------------------------------------------------------------
> summary estatistico: refere-se a uma análise descritiva dos dados que inclui medidas como média, mediana, desvio padrão, valores mínimos e máximos, entre outros.
- usa-se: 
    - ```df.describe()``` no pandas.

> distribuições: refere-se à forma como os dados estão distribuídos para cada variável. 
- Isso pode incluir:
    - histogramas:
        - usa-se:
            - ````sns.histplot()```` do seaborn, ````plt.hist()```` do matplotlib.
    - boxplots:
        - usa-se:
            - ````sns.boxplot()```` do seaborn, ````plt.boxplot()```` do matplotlib.
    - KDE plots (Kernel Density Estimation):
        - usa-se:
            - ````sns.kdeplot()```` do seaborn.

> correlações: medem quanto uma variável se relaciona com outra.
    > A matriz de correlação ````df.corr()```` mostra valores entre -1 e 1, indicando a força e direção da relação.
- +1: correlação positiva perfeita (ambas crescem juntas)
- 1: correlação negativa perfeita (uma cresce, outra diminui)
- 0: sem relação linear clara
- usa-se:
    - ````sns.heatmap()```` do seaborn para visualizar a matriz de correlação.

> contagem por categoria: refere-se à contagem de ocorrências para cada categoria em variáveis categóricas.
- usa-se:
    - ````df['categoria'].value_counts()```` no pandas.

> percentuais de nulos: refere-se à proporção de valores ausentes em cada coluna do dataset.
- usa-se:
    - ````df.isnull().mean() * 100```` no pandas. -> lê-se como "percentual de valores nulos por coluna".


> checar duplicatas e formato de datas: refere-se à verificação de registros duplicados no dataset e à validação do formato das colunas de data.
- importa pois:
    - duplicatas podem ganhar peso indevido.
    - datas mal formatadas impedem cálculos corretos de recência, idade da conta, etc. 
        - exemplo: 
            - “2025/01/02”, “02-01-2025” e “01-02-25” podem representar coisas diferentes dependendo do padrão adotado (americano vs brasileiro).
            - converter signup_date/last_login para datetime.
- usa-se para duplicatas:

    ```python
     # para contar duplicatas.
    df.duplicated().sum()

     # para visualizar duplicatas.
    df[df.duplicated()]

    # para remover duplicatas.
    df.drop_duplicates()

    # para remover duplicatas parcialmente
    df = df.drop_duplicates(subset=['customer_id', 'signup_date'])
    ```

- usa-se para datas:
    ```python
    # Converter coluna de string para datetime
    df['data'] = pd.to_datetime(df['data'], errors='coerce', dayfirst=True)
    
    # Checar valores inválidos (que viraram NaT)
    df['data'].isna().sum()
    
    # Extrair partes úteis da data
    df['ano'] = df['data'].dt.year
    df['mes'] = df['data'].dt.month
    df['dia_semana'] = df['data'].dt.day_name()

    # signup_date/last_login
    df['signup_date'] = pd.to_datetime(df['signup_date'], errors='coerce', dayfirst=True)
    df['last_login'] = pd.to_datetime(df['last_login'], errors='coerce', dayfirst=True)

    ```
----------------------------------------------------------------------------------------------------------------------------------------------------------------
2. Limpeza
    - remover duplicatas (total ou parcial por customer_id + signup_date).
    - converter signup_date/last_login para datetime.
    - criar coluna boolean indicando se last_login é nulo.
    - exemplo: 
    ```python
    df['no_login'] = df['last_login'].isna() # isna() retorna True para valores nulos
    ```
    <br><br>
3. Features temporais
    > recency = (today - last_login).days (usar um referência fixa para reprodutibilidade).
    - Conceito: Mede a quantidade de dias que se passaram desde a última interação (neste caso, o último login) do usuário até a data de hoje.
    - Interpreção: Um valor baixo indica que o usuário fez login recentemente e está engajado.Um valor alto indica que o usuário está inativo ou pode estar em risco de churn (abandono).
    uso: 
    ```python
    df['recency'] = (pd.Timestamp("2025-10-01") - df['last_login']).dt.days # exemplo com data fixa
    ```
    <br><br>
    > account_age_days = (today - signup_date).days.
    - Conceito: Mede a idade total da conta do usuário em dias, desde a data de cadastro (signup_date) até hoje.
    - Interpretação: Um valor alto indica que o usuário é um cliente antigo, o que pode sugerir lealdade. Um valor baixo indica um cliente novo, que pode estar em fase de avaliação do serviço.
    uso: 
    ```python
    df['account_age_days'] = (pd.Timestamp("2025-10-01") - df['signup_date']).dt.days # exemplo com data fixa
    ```
    <br><br>
    > activity_rate = num_transactions / max(1, account_age_days/30).
    - Conceito: Mede a frequência média de transações do usuário por mês, normalizando pelo tempo que a conta está ativa.
    - Detalhes da fórmula:
        - num_transactions: O número total de transações feitas pelo usuário.
        - account_age_days/30: Converte a idade da conta em dias para a idade da conta em meses.
        max(1, ...): É uma técnica de proteção contra divisão por zero ou por um período de tempo muito pequeno. Garante que o denominador seja pelo menos 1, o que é crucial para contas criadas há menos de um mês (para evitar uma taxa de atividade inflacionada ou divisão por zero).
    - Interpretação: Um valor alto indica um usuário ativo que realiza transações regularmente. Um valor baixo pode indicar um usuário inativo ou com baixo engajamento.
    uso: 
    ```python
    # Calcula a idade em meses, garantindo que o divisor seja no mínimo 1
    age_months = df_usuarios['account_age_days'] / 30
    divisor = np.maximum(1, age_months)

    df_usuarios['activity_rate'] = df_usuarios['num_transactions'] / divisor
    ```

4. Tratamento de missing
    > income: imputar com median + flag is_income_missing.
    - imputação:
        - 
    > last_login: imputar com signup_date (ou grande recency) + flag.
    > review_text: preencher "" + flag.

5. Outliers
    - truncamento (winsorize) ou log-transform para income e avg_transaction; criar versão transformada.
6. Encoding
    - categóricas: One-Hot para device/product; target/mean-encoding ou frequency-encoding para country (avaliar leakage).
    - texto: TF-IDF (unigram/bigram) limitando vocab (top k) ou usar hash vectorizer.
7. Pipeline e seleção
    - montar sklearn ColumnTransformer + Pipeline (imputer, scaler, encoders, text vect).
    - reduzir dimensionalidade texto com TruncatedSVD se usar TF-IDF.
8. Validação e modelagem
    - split temporal: usar cutoff por signup_date (treino até T, teste após T) ou holdout estratificado respeitando tempo.
    - baseline model: LogisticRegression (class_weight='balanced') e/ou XGBoost.
    - métricas: ROC AUC, Precision@k (ex.: top 10%), e matriz de confusão; reportar recall/precision para classe positiva.
9. Interpretabilidade
    - features mais importantes (coeficientes ou SHAP).
    - avaliar ganho por tipo de feature (numéricas vs texto vs categóricas).

Entregáveis esperados:

- Notebook (feature_engineering2.ipynb) com código organizado, células comentadas e resultados (gráficos/tabelas).
- Modelo salvo (pickle) e arquivo com feature importances.
- Breve relatório (markdown no notebook) com:
    - resumo das transformações,
    - métricas finais no conjunto teste,
    - duas recomendações de próximas melhorias.

Critério mínimo de aceitação:

- Pipeline reproduzível (pode rodar a célula final e gerar métricas).
- ROC AUC >= 0.70 ou explicação curta se não atingir (p.ex., sinal fraco).
- Documentação das decisões de imputação e tratamento de outliers.

Dicas rápidas de implementação:

- use sklearn.compose.ColumnTransformer, sklearn.pipeline.Pipeline.
- para texto: sklearn.feature_extraction.text.TfidfVectorizer + TruncatedSVD.
- para avaliação temporal: sort by signup_date e use TimeSeriesSplit ou cutoff manual.
- fixe "today" com uma data constante (ex.: pd.Timestamp("2025-10-01")) para reprodutibilidade.

- o pandas é para manipulação de dados
- a numpy é para trabalharmos com algumas operações matematicas
- o dataframe serve para organizar e manipular dados tabulares, como planilhas ou tabelas de banco de dados

In [3]:
import pandas as pd # utilização do pandas para manipulação de dados
import numpy as np # utilização do numpy para operações numéricas
from pandas import DataFrame # importação específica do DataFrame do pandas
import seaborn as sns # utilização do seaborn para visualização de dados
import matplotlib.pyplot as plt # utilização do matplotlib para criação de gráficos
print("Bibliotecas importadas com sucesso.", np, pd, DataFrame, sns, plt)

Bibliotecas importadas com sucesso. <module 'numpy' from 'c:\\Users\\joaop\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages\\numpy\\__init__.py'> <module 'pandas' from 'c:\\Users\\joaop\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages\\pandas\\__init__.py'> <class 'pandas.core.frame.DataFrame'> <module 'seaborn' from 'c:\\Users\\joaop\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages\\seaborn\\__init__.py'> <module 'matplotlib.pyplot' from 'c:\\Users\\joaop\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages\\matplotlib\\pyplot.py'>


In [4]:
data = pd.read_csv("datasets/train_sample.csv")
data.head() # Display the first few rows of the dataset

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,89489,3,1,13,379,2017-11-06 15:13:23,NaN,0
1,204158,35,1,13,21,2017-11-06 15:41:07,2017-11-07 08:17:19,1
2,3437,6,1,13,459,2017-11-06 15:42:32,NaN,0
3,167543,3,1,13,379,2017-11-06 15:56:17,NaN,0
4,147509,3,1,13,379,2017-11-06 15:57:01,NaN,0
